# New York Times News Summarization
Sirut Buasai, sbausai2@wpi.edu <br>
Jason Dykstra, jpdykstra@wpi.edu <br>
Adam Yang ayang@wpi.edu
## Libraries

In [6]:
# Data Processing
import pandas as pd
from sklearn.model_selection import train_test_split

# Text Processing
import nltk
import re
from nltk.corpus import stopwords
from nltk import pos_tag_sents
import spacy
NER = spacy.load("en_core_web_sm")

# Model Building
import tensorflow as tf
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer


nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Data Pre-processing
### Initial Data Inspection

In [2]:
raw_data_path = 'NYT_Dataset.csv'
raw_data = pd.read_csv(raw_data_path)
print(f'dataframe shape: {raw_data.shape}')
raw_data

dataframe shape: (106506, 7)


,Unnamed: 0,ID,title,topic,abstract,Date,keywords
0,0,nyt://article/178801fe-4679-5f12-985f-8344a86e...,"In Reversal, Pakistan Welcomes Outside Help Wi...",Foreign,Pakistan’s ambassador to the U.S. said his gov...,2008-01-01 05:00:00+00:00,['Assassinations and Attempted Assassinations'...
1,1,nyt://article/21acedcb-a7f6-5131-99cf-d3a47e33...,Fighting Intensifies After Election in Kenya,Foreign,"Kenya sank deeper into trouble, with a curfew ...",2008-01-01 05:00:00+00:00,"['Kenya', 'Demonstrations and Riots', 'Odinga,..."
2,2,nyt://article/357b5429-a9f8-5d33-a5eb-c013a201...,Israel: Olmert Curbs Settlements,Foreign,Prime Minister Ehud Olmert has sent a letter t...,2008-01-01 05:00:00+00:00,['West Bank']
3,3,nyt://article/619ca4ea-50e4-59e4-97bb-f206502c...,Gay Muslims Pack a Dance Floor of Their Own,Foreign,The monthly club night known as Gayhane is an ...,2008-01-01 05:00:00+00:00,"['Homosexuality', 'Islam', 'IMMIGRATION AND RE..."
4,4,nyt://article/73c49a5a-bcf1-5b8f-a15a-98d29003...,Iraqi Revelers Embrace the New Year,Foreign,"But even as partygoers embraced the New Year, ...",2008-01-01 05:00:00+00:00,"['ARMAMENT, DEFENSE AND MILITARY FORCES', 'Iraq']"
...,...,...,...,...,...,...,...
106501,106501,nyt://article/7f2445bc-7094-52f2-866e-2dd09f7e...,BAFTA Suspends Award for Actor Noel Clarke Ami...,Foreign,The British actor and director has been accuse...,2021-04-30 13:52:12+00:00,"['Sexual Harassment', 'Actors and Actresses', ..."
106502,106502,nyt://article/509b840b-52b0-54ca-8e0e-21c2c8fd...,The Bureaucrat From Buffalo Who Pushed Somalia...,Foreign,"His bid to stay in office an extra two years, ...",2021-04-30 16:07:49+00:00,"['Mohamed, Mohamed Abdullahi', 'Mogadishu (Som..."
106503,106503,nyt://interactive/92d8a9aa-9deb-5ff0-9fc6-6661...,What to Know About the Census Data,U.S.,The count reflects the slowest population grow...,2021-04-30 16:40:05+00:00,"['Census', 'Population', 'United States', 'Rac..."
106504,106504,nyt://article/b5a25627-cf5f-528f-a69c-afcfa681...,"After 500 Years, an Ancient Bronze Hand Is Rej...",Foreign,Researchers still don’t know how a finger belo...,2021-04-30 16:46:47+00:00,"['Art', 'Roman Civilization', 'Capitoline Muse..."


### Clean Data

In [8]:
# FUNCTION FOR PARTS-OF-SPEECH PROCESSING
def pos_processing(abstract_tokens):
    # identify parts-of-speech; pos_tag requires list of tokens
    abstract_pos = pd.Series(pos_tag_sents(abstract_tokens))
    
    
    # create attention mask(s) using parts-of-speech
    nounsList = ['NN', 'NNS', 'NNP', 'NNPS'] # nouns to pay attention to
    noun_attention_mask = abstract_pos.apply(
                            lambda words: [1 if pair[1] in nounsList else 0 for pair in words])

    return noun_attention_mask

In [9]:
# FUNCTIONS FOR NER PROCESSING
def create_ne_attn_mask(entities):
    # extract named entities
    named_entities = entities.ents
    # initialize attention mask
    attn_mask = [0 for i in range(len(entities))]

    # assign 1's to positions in attn mask corresponding to named-entity positions
    for entity in named_entities:
        attn_mask[entity.start: entity.end] = [1 for i in range(entity.start,entity.end)]

    return attn_mask


def ner_processing(abstract_sentences):
    # identify entities for each abstract
    # NOTE: this is supposedly faster if we use nlp.pipe, but we can look into that tomorrow ...
    abs_entities = pd.Series(abstract_sentences.apply(lambda sentence: NER(sentence)))

    # create attention mask(s) using the NE's
    ne_attention_mask = abs_entities.apply(lambda entities: create_ne_attn_mask(entities))

    return ne_attention_mask


In [10]:
# GENERAL DATA-PREPROCESSING FUNCTION
def preprocess_data(df):
    # drop unnecessary columns
    unused_columns = ['Unnamed: 0', 'Date', 'ID', 'topic', 'keywords']
    df.drop(unused_columns, axis=1, inplace=True, errors='ignore')

    # drop dupliates and nan rows
    df.drop_duplicates(subset=['abstract'], inplace=True)
    df.dropna(axis=0, inplace=True)

    # convert texts to lowercase
    df['abstract'] = df['abstract'].str.lower()
    df['title'] = df['title'].str.lower()

    # strip special characters
    df['cleaned_title'] = df['title'].apply(lambda string: re.sub("[^a-zA-Z0-9 ]+", '', string))
    df['cleaned_abstract'] = df['abstract'].apply(lambda string: re.sub("[^a-zA-Z0-9 ]+", '', string))

    # convert string to list
    df['cleaned_title'] = df['cleaned_title'].apply(lambda string: string.split())
    df['cleaned_abstract'] = df['cleaned_abstract'].apply(lambda string: string.split())

    # remove stopwords
    stop_words = stopwords.words('english')
    df['cleaned_title'] = df['cleaned_title'].apply(lambda sentence: [word for word in sentence if word not in (stop_words)])
    df['cleaned_abstract'] = df['cleaned_abstract'].apply(lambda sentence: [word for word in sentence if word not in (stop_words)])

    # generate noun attention mask from parts-of-speech
    df['noun_attn_mask'] = pos_processing(df['cleaned_abstract'])

    # convert list back to string
    df['cleaned_title'] = df['cleaned_title'].apply(lambda sentence: " ".join(word for word in sentence))
    df['cleaned_abstract'] = df['cleaned_abstract'].apply(lambda sentence: " ".join(word for word in sentence))
    
    # identify named-entities; NER requires string sentences
    df['ne_attention_mask'] = ner_processing(df['cleaned_abstract'])
    
    # add start and end tokens for title
    df['cleaned_title'] = df['cleaned_title'].apply(lambda text: '_START_ ' + text + ' _END_')

    return df

cleaned_data = preprocess_data(raw_data)
print(f'dataframe shape: {cleaned_data.shape}')
cleaned_data

dataframe shape: (103825, 6)


,title,abstract,cleaned_title,cleaned_abstract,noun_attn_mask,ne_attention_mask
0,"in reversal, pakistan welcomes outside help wi...",pakistan’s ambassador to the u.s. said his gov...,_START_ reversal pakistan welcomes outside hel...,pakistans ambassador us said government would ...,"[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
1,fighting intensifies after election in kenya,"kenya sank deeper into trouble, with a curfew ...",_START_ fighting intensifies election kenya _END_,kenya sank deeper trouble curfew imposed kisum...,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,israel: olmert curbs settlements,prime minister ehud olmert has sent a letter t...,_START_ israel olmert curbs settlements _END_,prime minister ehud olmert sent letter defense...,"[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,gay muslims pack a dance floor of their own,the monthly club night known as gayhane is an ...,_START_ gay muslims pack dance floor _END_,monthly club night known gayhane alltoorare op...,"[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,iraqi revelers embrace the new year,"but even as partygoers embraced the new year, ...",_START_ iraqi revelers embrace new year _END_,even partygoers embraced new year surge attack...,"[0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1]","[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0]"
...,...,...,...,...,...,...
106501,bafta suspends award for actor noel clarke ami...,the british actor and director has been accuse...,_START_ bafta suspends award actor noel clarke...,british actor director accused sexual assault ...,NaN,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
106502,the bureaucrat from buffalo who pushed somalia...,"his bid to stay in office an extra two years, ...",_START_ bureaucrat buffalo pushed somalia brin...,bid stay office extra two years without electi...,NaN,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
106503,what to know about the census data,the count reflects the slowest population grow...,_START_ know census data _END_,count reflects slowest population growth since...,NaN,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
106504,"after 500 years, an ancient bronze hand is rej...",researchers still don’t know how a finger belo...,_START_ 500 years ancient bronze hand rejoined...,researchers still dont know finger belonging c...,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


### Declare Maximum Title and Abstract Length

In [ ]:
MAX_TITLE_LEN = 100
MAX_ABSTRACT_LEN = 20

### Split Training and Testing Set

In [ ]:
abs_tr, abs_te, ttl_tr, ttl_te = train_test_split(
  cleaned_data['cleaned_abstract'],
  cleaned_data['cleaned_title'],
  test_size=0.1,
  random_state=0,
  shuffle=True)

### Tokenize Title and Abstracts

In [ ]:
def tokenize_text(train_set, test_set):
  # prepare a tokenizer for reviews on training data
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(list(train_set))

  # convert text sequences into integer sequences
  train_set = tokenizer.texts_to_sequences(train_set) 
  test_set = tokenizer.texts_to_sequences(test_set)

  # padding zero up to maximum length
  train_set = pad_sequences(train_set, maxlen=MAX_ABSTRACT_LEN, padding='post') 
  test_set = pad_sequences(test_set, maxlen=MAX_ABSTRACT_LEN, padding='post')

  # get vector size
  vocab_size = len(tokenizer.word_index)+1

  return tokenizer, train_set, test_set, vocab_size

abs_tokenizer, abs_tr, abs_te, abs_size = tokenize_text(abs_tr, abs_te)
ttl_tokenizer, ttl_tr, ttl_te, ttl_size = tokenize_text(ttl_tr, ttl_te)

## Build LSTM Model

In [ ]:
from keras import backend as K 
K.clear_session() 
latent_dim = 500 

# encoder 
encoder_inputs = tf.keras.layers.Input(shape=(MAX_ABSTRACT_LEN,)) 
enc_emb = tf.keras.layers.Embedding(abs_size, latent_dim,trainable=True)(encoder_inputs) 

# LSTM 1 
encoder_lstm1 = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

# LSTM 2 
encoder_lstm2 = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# LSTM 3 
encoder_lstm3 = tf.keras.layers.LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# set up the decoder. 
decoder_inputs = tf.keras.layers.Input(shape=(None,)) 
dec_emb_layer = tf.keras.layers.Embedding(ttl_size, latent_dim, trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

# LSTM using encoder_states as initial state
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

# dense layer
decoder_dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(ttl_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_outputs) 

# define the model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

### Train the Model

In [ ]:
# initialize model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# initialize early stopping when validation loss increases
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# train model
history = model.fit([abs_tr, ttl_tr[:,:-1]], ttl_tr.reshape(ttl_tr.shape[0], ttl_tr.shape[1], 1)[:,1:],
                    epochs=50,
                    callbacks=[es],
                    batch_size=512,
                    validation_data=([abs_te,ttl_te[:,:-1]], ttl_te.reshape(ttl_te.shape[0], ttl_te.shape[1], 1)[:,1:]))
